# Proyecto sobre Sistemas Recomendadores Basados en Contenido

### Objetivo
El objetivo de este proyecto es implementar un sistema de recomendación basado en contenido, que nos permita recomendar los mejores documentos para un cliente, mediante el algoritmo de clasificación KNN. 

### Metodología
1. Estudiar los modelos basados en el contenido en el material de clase.
2. Crear un software que reciba un archivo de texto plano con extensión .txt, que contenga el conjunto de posibles documentos a recomendar al usuario final. Cada documento viene representado en una línea del archivo.
3. El software debe proporcionar como salida lo siguiente para cada documento:
    - Índice del término.
    - Término.
    - TF.
    - IDF.
    - TF-IDF.
    - Similaridad coseno entre cada par de documentos.

### Entregables
- Enlace a repositorio GitHub público con el código fuente del sistema recomendador implementado, incluyendo un fichero README.md del repositorio, que contenga la descripción del código desarrollado y un ejemplo de uso.
- Informe en PDF describiendo el análisis realizado en varios ejemplos y las conclusiones extraídas. 

###### Importamos los paquetes necesarios para realizar el proyecto

In [1]:
import numpy as np
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
import string

# Imports
import pandas as pd

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.es import Spanish

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

import pickle

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

nltk.download('stopwords')
nltk.download('punkt')

from IPython.display import display, HTML
from sklearn.feature_extraction.text import CountVectorizer
# https://www.aurigait.com/blog/recommendation-system-using-knn/
# https://www.analyticsvidhya.com/blog/2020/08/recommendation-system-k-nearest-neighbors/
# https://www.mygreatlearning.com/blog/nltk-tutorial-with-python/
# https://www.datacamp.com/community/tutorials/recommender-systems-python
# https://stackabuse.com/python-for-nlp-tokenization-stemming-and-lemmatization-with-spacy-library/
# https://es.stackoverflow.com/questions/331602/c%C3%B3mo-lematizar-en-espa%C3%B1ol-una-string-con-nltk
# https://medium.com/@jae.finger/building-a-marijuana-strain-recommender-system-bef3f66bb2d3
# https://scikit-learn.org/stable/modules/neighbors.html

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jdani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jdani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##### Leemos el documento  que, en este caso, es un texto plano. Añadimos una columna que indica el género simplemente para añadir más información.

In [2]:
file = open("Documentos.txt", "r", encoding='utf8')
lines = file.readlines()

datos = {'Género':['Salud', 'Salud', 'Salud', 'Cine', 'Ciencia', 'Salud', 'Salud'],
        'Documentos':lines}
df = pd.DataFrame(data = datos)
df.head(10)

,Género,Documentos
0,Salud,Los grupos musculares se ejercitan en función ...
1,Salud,Existen dos tipos de actividades física que se...
2,Salud,Los deportes más practicados a nivel mundial s...
3,Cine,Las películas con más Oscars de la historia so...
4,Ciencia,El sol es una estrella que irradia calor y ene...
5,Salud,Practicar deporte es altamente recomendado par...
6,Salud,Se han realizado varios estudios reflejando la...


##### Eliminamos el elemento '.\n' que aparece al final de cada registro. Este elemento simplemente indica el salto de línea, pero al leer el documento .txt también se ha interpretado como texto.

In [3]:
df_copy = df.copy()
for i,x in enumerate(df_copy['Documentos']):
    # Indicamos el [:-2] para elminar la cadena '\n' del final de cada documento   
    df_copy['Documentos'][i] = df_copy['Documentos'][i][:-2]
    print(df_copy['Documentos'][i])

df['Documentos'] = df_copy['Documentos']

Los grupos musculares se ejercitan en función del ejercicio específico que se realice, por ejemplo, si quieres realizar un ejercicio de alta intensidad, optarás por uno anaeróbico y viceversa
Existen dos tipos de actividades física que se suelen practicar, los ejercicios aeróbicos y los ejercicios anaeróbicos
Los deportes más practicados a nivel mundial son el fútbol y el baloncesto, donde se fomenta el compañerismo y el respeto
Las películas con más Oscars de la historia son Titanic, BenHur y El Señor de los Anillos
El sol es una estrella que irradia calor y energía por toda la vía lactea. Es parte fundamental del sistema solar
Practicar deporte es altamente recomendado para mejorar la salud física y mental, según han publicado varias revistas y organismos
Se han realizado varios estudios reflejando la importancia del ejericio como medida para contrarrestar el estrés


###### Creamos una columna nueva añadiendo el género a cada documento, para añadir mas información para el entrenamiento posterior. Además convertimos a minúsculas todas las palabras del documento.


In [4]:
df['Genero_+_documento'] = df['Género']+ ' ' + df['Documentos']

df['Genero_+_documento'] = df['Genero_+_documento'].apply(lambda x: str.lower(x))

##### Definimos una función que realiza lo siguiente:
- Eliminamos los signos de puntuación. 
- Tokenizamos cada documento.
- Nos devuelve una lista de palabras a partir del documento que se le pase

Aplicamos esta función a cada documento y creamos una nueva columna para almacenar los documentos tokenizados.

In [5]:
# https://www.mygreatlearning.com/blog/nltk-tutorial-with-python/
# https://datagy.io/python-remove-punctuation-from-string/
    
### Primero tokenizamos los 4 primeros documentos
def tokenize_words(text):
    sin_puntuacion = text.translate(str.maketrans('', '', string.punctuation))
    tokenized_text=word_tokenize(sin_puntuacion)
    print(tokenized_text)
    print(type(tokenized_text))
    
    return tokenized_text

# df_tokenized = df.copy()
df['Words_tokenized'] = df['Genero_+_documento'].apply(lambda x: tokenize_words(x))
    


['salud', 'los', 'grupos', 'musculares', 'se', 'ejercitan', 'en', 'función', 'del', 'ejercicio', 'específico', 'que', 'se', 'realice', 'por', 'ejemplo', 'si', 'quieres', 'realizar', 'un', 'ejercicio', 'de', 'alta', 'intensidad', 'optarás', 'por', 'uno', 'anaeróbico', 'y', 'viceversa']
<class 'list'>
['salud', 'existen', 'dos', 'tipos', 'de', 'actividades', 'física', 'que', 'se', 'suelen', 'practicar', 'los', 'ejercicios', 'aeróbicos', 'y', 'los', 'ejercicios', 'anaeróbicos']
<class 'list'>
['salud', 'los', 'deportes', 'más', 'practicados', 'a', 'nivel', 'mundial', 'son', 'el', 'fútbol', 'y', 'el', 'baloncesto', 'donde', 'se', 'fomenta', 'el', 'compañerismo', 'y', 'el', 'respeto']
<class 'list'>
['cine', 'las', 'películas', 'con', 'más', 'oscars', 'de', 'la', 'historia', 'son', 'titanic', 'benhur', 'y', 'el', 'señor', 'de', 'los', 'anillos']
<class 'list'>
['ciencia', 'el', 'sol', 'es', 'una', 'estrella', 'que', 'irradia', 'calor', 'y', 'energía', 'por', 'toda', 'la', 'vía', 'lactea', '

##### Definimos una función que filtra cada documento tokenizado para eliminar las stopwords. Creamos otra variable dentro del dataset para almacenar  la nueva lista sin los stopwords

In [6]:
stopword_es = nltk.corpus.stopwords.words('spanish')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword_es]
    return text

df['Sin_stopwords'] = df['Words_tokenized'].apply(lambda x: remove_stopwords(x))

In [7]:
df.head()

,Género,Documentos,Genero_+_documento,Words_tokenized,Sin_stopwords
0,Salud,Los grupos musculares se ejercitan en función ...,salud los grupos musculares se ejercitan en fu...,"[salud, los, grupos, musculares, se, ejercitan...","[salud, grupos, musculares, ejercitan, función..."
1,Salud,Existen dos tipos de actividades física que se...,salud existen dos tipos de actividades física ...,"[salud, existen, dos, tipos, de, actividades, ...","[salud, existen, dos, tipos, actividades, físi..."
2,Salud,Los deportes más practicados a nivel mundial s...,salud los deportes más practicados a nivel mun...,"[salud, los, deportes, más, practicados, a, ni...","[salud, deportes, practicados, nivel, mundial,..."
3,Cine,Las películas con más Oscars de la historia so...,cine las películas con más oscars de la histor...,"[cine, las, películas, con, más, oscars, de, l...","[cine, películas, oscars, historia, titanic, b..."
4,Ciencia,El sol es una estrella que irradia calor y ene...,ciencia el sol es una estrella que irradia cal...,"[ciencia, el, sol, es, una, estrella, que, irr...","[ciencia, sol, estrella, irradia, calor, energ..."


##### Aplicamos stemmización, es decir, sacamos la raíces de las palabras y las almacenamos en otra variable.

In [8]:
def spanish_stemmer(x):
    spanish_stemmer_ = SnowballStemmer(language = 'spanish')
    for i in range(len(x)):
#         print(i)
        for l in range(len(x[i])):
#             print(l)
            x[i][l] = spanish_stemmer_.stem(x[i][l])
#             print(x[i])
    return x
    
df['words_stemmed'] = spanish_stemmer(df['Sin_stopwords'])

In [9]:
df.head()

,Género,Documentos,Genero_+_documento,Words_tokenized,Sin_stopwords,words_stemmed
0,Salud,Los grupos musculares se ejercitan en función ...,salud los grupos musculares se ejercitan en fu...,"[salud, los, grupos, musculares, se, ejercitan...","[salud, grup, muscular, ejercit, funcion, ejer...","[salud, grup, muscular, ejercit, funcion, ejer..."
1,Salud,Existen dos tipos de actividades física que se...,salud existen dos tipos de actividades física ...,"[salud, existen, dos, tipos, de, actividades, ...","[salud, exist, dos, tip, activ, fisic, suel, p...","[salud, exist, dos, tip, activ, fisic, suel, p..."
2,Salud,Los deportes más practicados a nivel mundial s...,salud los deportes más practicados a nivel mun...,"[salud, los, deportes, más, practicados, a, ni...","[salud, deport, practic, nivel, mundial, futbo...","[salud, deport, practic, nivel, mundial, futbo..."
3,Cine,Las películas con más Oscars de la historia so...,cine las películas con más oscars de la histor...,"[cine, las, películas, con, más, oscars, de, l...","[cin, pelicul, oscars, histori, titanic, benhu...","[cin, pelicul, oscars, histori, titanic, benhu..."
4,Ciencia,El sol es una estrella que irradia calor y ene...,ciencia el sol es una estrella que irradia cal...,"[ciencia, el, sol, es, una, estrella, que, irr...","[cienci, sol, estrell, irradi, calor, energ, t...","[cienci, sol, estrell, irradi, calor, energ, t..."


###### Calculamos los valores de TF, IDF y TF-IDF

In [17]:
#Definir un objeto vectorial TF-IDF. 
tfidf = TfidfVectorizer(analyzer = 'word',lowercase = True, stop_words = stopword_es)

# Construimos la matriz TF-IDF 
tfidf_matrix = tfidf.fit_transform(df['words_stemmed'].astype('U'))

#MOstramos las dimensiones de la matriz  TFIDF
tfidf_matrix.shape

print(tfidf_matrix)

  (0, 68)	0.23955658657259687
  (0, 4)	0.19885253980650688
  (0, 42)	0.23955658657259687
  (0, 33)	0.23955658657259687
  (0, 3)	0.23955658657259687
  (0, 51)	0.19885253980650688
  (0, 49)	0.23955658657259687
  (0, 59)	0.23955658657259687
  (0, 15)	0.23955658657259687
  (0, 50)	0.23955658657259687
  (0, 20)	0.23955658657259687
  (0, 16)	0.39770507961301377
  (0, 27)	0.23955658657259687
  (0, 17)	0.23955658657259687
  (0, 40)	0.23955658657259687
  (0, 30)	0.23955658657259687
  (0, 56)	0.1292685122561784
  (1, 1)	0.3024908939685984
  (1, 47)	0.21462633135822845
  (1, 62)	0.3024908939685984
  (1, 25)	0.2510934196992658
  (1, 0)	0.3024908939685984
  (1, 63)	0.3024908939685984
  (1, 14)	0.3024908939685984
  (1, 24)	0.3024908939685984
  :	:
  (4, 61)	0.5163977794943222
  (4, 9)	0.2581988897471611
  (5, 43)	0.2919126272373849
  (5, 55)	0.2919126272373849
  (5, 66)	0.24231254985824804
  (5, 48)	0.2919126272373849
  (5, 57)	0.2919126272373849
  (5, 38)	0.2919126272373849
  (5, 37)	0.291912627237

##### Creamos nuestro dataset con los pesos asignados a cada palabra stemmizada. Este dataset es el que usaremos para entrenar nuestro modelo.

In [25]:
# dtm = tf.fit_transform(df[‘Criteria’].values.astype(‘U’))
#### para la versión del paquete más actual utilizamos esta línea, ya que usa get_feature_name_out
# dtm = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf.get_feature_names_out()) 


dtm = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf.get_feature_names_out())
print('TF-IDF')
print('==========================')
dtm

TF-IDF


,activ,aerob,alt,alta,anaerob,anill,baloncest,benhur,calor,cienci,...,si,sistem,sol,suel,tip,titanic,tod,vari,via,vicevers
0,0.000000,0.000000,0.000000,0.239557,0.198853,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.239557,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.239557
1,0.302491,0.302491,0.000000,0.000000,0.251093,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.302491,0.302491,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.343327,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.353553,0.000000,0.353553,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.353553,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.258199,0.258199,...,0.000000,0.258199,0.516398,0.000000,0.000000,0.000000,0.258199,0.000000,0.258199,0.000000
5,0.000000,0.000000,0.291913,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.242313,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.281924,0.000000,0.000000


##### Calculamos la frecuencia de las palabras

In [23]:
# https://towardsdatascience.com/a-gentle-introduction-to-calculating-the-tf-idf-values-9e391f8a13e5

## Obtenemos la cuenta de las palabras estemmizadas
## ================================
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['words_stemmed'].astype('U'))
terms = count_vect.get_feature_names_out()
terms

## Definimos una función a la que le pasamos la frecuencia de las palabras 
## y las palabras
## ================================
def dtm2df(wm, feat_names):
    
    # create an index for each row
    doc_names = ['Doc{:d}'.format(idx) for idx, _ in enumerate(wm)]
    df = pd.DataFrame(data=wm.toarray(), index=doc_names,
                      columns=feat_names)
    return(df)

print('Frecuencia de las palabras TF')
print('===============================')
df_count = dtm2df(X_train_counts ,terms)
display(HTML(df_count.to_html()))

Frecuencia de las palabras TF


,activ,aerob,alt,alta,anaerob,anill,baloncest,benhur,calor,cienci,cin,compañer,contrarrest,deport,dos,ejempl,ejercici,ejercit,ejerici,energ,especif,estrell,estres,estudi,exist,fisic,foment,funcion,fundamental,futbol,grup,histori,import,intens,irradi,lacte,med,mejor,mental,mundial,muscular,nivel,optaras,organ,oscars,part,pelicul,practic,public,quier,realic,realiz,recomend,reflej,respet,revist,salud,segun,señor,si,sistem,sol,suel,tip,titanic,tod,vari,via,vicevers
Doc0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,2,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1
Doc1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0
Doc2,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Doc3,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
Doc4,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,1,0,1,0
Doc5,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,2,1,0,0,0,0,0,0,0,0,1,0,0
Doc6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


##### Calculamos la frecuencia inversa de las palabras IDF

In [24]:
## Definimos una función a la que le pasamos el objeto de la clase tfidf y 
## las palabras, y nos devuelve el dataframe con el cálculo de IDF
## =============================================
def idf2df(wm, feat_names):
  
    # create an index for each row
    doc_names = ['Doc{:d}'.format(idx) for idx, _ in enumerate(wm)]
    df = pd.DataFrame(data=wm, index=[0],
                      columns=feat_names)
    return(df)


df_idf = idf2df(tfidf.idf_[:,np.newaxis].T ,terms)
print('Frecuencia inversa de las palabras IDF')
print('======================================')
display(HTML(df_idf.to_html()))

Frecuencia inversa de las palabras IDF


,activ,aerob,alt,alta,anaerob,anill,baloncest,benhur,calor,cienci,cin,compañer,contrarrest,deport,dos,ejempl,ejercici,ejercit,ejerici,energ,especif,estrell,estres,estudi,exist,fisic,foment,funcion,fundamental,futbol,grup,histori,import,intens,irradi,lacte,med,mejor,mental,mundial,muscular,nivel,optaras,organ,oscars,part,pelicul,practic,public,quier,realic,realiz,recomend,reflej,respet,revist,salud,segun,señor,si,sistem,sol,suel,tip,titanic,tod,vari,via,vicevers
0,2.386294,2.386294,2.386294,2.386294,1.980829,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,1.980829,2.386294,2.386294,1.980829,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,1.980829,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,1.693147,2.386294,2.386294,2.386294,1.980829,2.386294,2.386294,2.386294,2.386294,1.287682,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,2.386294,1.980829,2.386294,2.386294


##### Calculamos la similaridad del coseno entre cada par de documentos

In [32]:
# https://goodboychan.github.io/python/datacamp/natural_language_processing/2020/07/17/04-TF-IDF-and-similarity-scores.html

print('Similaridad del coseno')
print('======================================')
# Similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

data = {'Doc 1': cosine_sim[0],'Doc 2': cosine_sim[1],'Doc 3': cosine_sim[2],
       'Doc 4': cosine_sim[3],'Doc 5': cosine_sim[4],'Doc 6': cosine_sim[5],
       'Doc 7': cosine_sim[6]}
df = pd.DataFrame(data, index =['Doc 1','Doc 2','Doc 3','Doc 4',
                               'Doc 5','Doc 6','Doc 7'])
df

Similaridad del coseno


,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6,Doc 7
Doc 1,1.000000,0.270753,0.023949,0.0,0.0,0.040725,0.079752
Doc 2,0.270753,1.000000,0.082524,0.0,0.0,0.156720,0.029915
Doc 3,0.023949,0.082524,1.000000,0.0,0.0,0.177878,0.033954
Doc 4,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000
Doc 5,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000
Doc 6,0.040725,0.156720,0.177878,0.0,0.0,1.000000,0.126052
Doc 7,0.079752,0.029915,0.033954,0.0,0.0,0.126052,1.000000


###### Entrenamos el modelo de vecino más cercano utilizando la métrica de la similaridad del coseno

In [33]:
nn = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='cosine')

nn.fit(dtm.values) # Le ponemos values para que no aparezca el warning de los feature names
# https://stackoverflow.com/questions/69326639/sklearn-warning-valid-feature-names-in-version-1-0

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=1)

###### Definimos una función que pide por pantalla un documento a partir del cual queremos que el sistema nos recomiende. 
Le aplicaremos todos los pasos que hemos hecho anteriormente antes de hacer la predicción. Una vez hechos todos esos procesos, nos devuelve la recomendación, es decir, el documento que estaría más cercano al que nosotros hemos introducido.

In [34]:
def recomendar_documentos():
    peticion = input('Indique sobre qué quiere una recomendación: \n')
    
    tokenized = word_tokenize(str.lower(peticion))
    
    no_stop_word = remove_stopwords(tokenized)
    
    stemmed_words = [SnowballStemmer(language = 'spanish').stem(x) for x in no_stop_word]
    
    recomendacion = nn.kneighbors(tfidf.transform(stemmed_words).todense())
    
#     print(recomendacion)
    
    return df['Documentos'][recomendacion[1][0]]
#     return df['Documentos'][recomendacion[1][0]],df['Documentos'][recomendacion[1][1]], df['Documentos'][recomendacion[1][2]]

##### Ejemplos de documentos que leo para para que el sistema me recomiende sobre los que hay:
- Las películas del cine más importantes de la historia
- El sol es la estrella más importante de nuestro sistema solar pero es una más en el espacio.



In [ ]:
recomendar_documentos()